In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import pickle
from functools import partial

from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from catboost import CatBoostRegressor

In [3]:
#Сделаем настройки
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
plt.rcParams['figure.figsize'] = 12, 8
RANDOM_STATE=43

In [4]:
df = pd.read_csv('data2.csv')
df_train = df[df.is_train==1]

C:\Soft\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.shape

(70896, 43)

# Определим нужные нам функции

In [6]:
def mape(y_true, y_pred):
    """оценка MEAN ABSOLUTE PERCENTAGE ERROR, которая используется в соревновании"""
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


def score_model(model, X, y):
    """Оцениваем модель и данные на трех фолдах. Выводим средюю оценку, и оценки на фолдах"""
    kf = KFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
    cv_results = cross_val_score(
        model, X, y, cv=kf, n_jobs=-1, scoring=make_scorer(mape))
    print("#"*100)
    print(model)
    print("#"*100)
    print(f'mean_result: {np.mean(cv_results)} ({cv_results})')
    print("#"*100)


def make_submit(model, X_test, submit_file='predict.csv'):
    """запишем предсказание модели в файл, готовый к отправке на kaggle"""
    pred = np.round(model.predict(X_test))
    answer = pd.Series(pred, index=df[~df.is_train].sell_id, name='price')
    answer.to_csv(submit_file, index_label='sell_id')


def prepare_data(categorical_columns: list, numerical_columns: list):
    """Вытащим требуемые данные из переменной df
    категориальные применим dummy кодирование
    числовые Шкалирование"""
    X = df.loc[:, categorical_columns+numerical_columns+['is_train']]

    X = pd.get_dummies(X, columns=categorical_columns)
    X[numerical_columns] = StandardScaler().fit_transform(X[numerical_columns])
    
    X_train = X[X.is_train == 1].drop('is_train', axis=1)
    X_test = X[X.is_train == 0].drop('is_train', axis=1)

    y = df[df.is_train].price

    return X_train, X_test, y

# Подготовим тренировочные и тестовые наборы 

Состав колонок как в предыдущей части

In [7]:
cat_cols = ['bodyType', 'brand', 'fuelType', 'color', 'model_name',
            'vehicleTransmission', 'vendor', 'ПТС', 'Владельцы', 'Привод', 'Руль'] 
num_cols = ['productionDate', 'numberOfDoors',
            'enginePower_log', 'mileage_log', 'diff_date_model_production']

In [8]:
X_train, X_test, y = prepare_data(cat_cols, num_cols)

# Подбор параметров модели

Лучший результат показал случайный лес. Давайте попробуем как будет он себя вести, при изменени его деволтных параметров

Хотел взять критерий MAE для  модели, но она очень долго тренирутеся, даже если поставить n_estimators=10

In [9]:
rf_model=RandomForestRegressor(random_state=RANDOM_STATE, n_estimators=100)

In [10]:
param_grid = { 
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,6, None],
}

In [12]:
CV_rf = GridSearchCV(rf_model, param_grid, cv=3)


In [ ]:
%%time
CV_rf.fit(X_train, y)

In [ ]:
CV_rfc.best_params_